In [91]:
import os
import openai

In [92]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [93]:
from git import Repo

In [94]:
from pathlib import Path

In [95]:
pwd

'/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator'

In [96]:
PATH_TO_BLOG_REPO = Path("/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/.git")

In [97]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [98]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [99]:
PATH_TO_CONTENT

PosixPath('/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/content')

In [100]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [101]:
def update_blog(commit_message='Updates blog'):
    # GitPython -- Repo locations
    repo = Repo(PATH_TO_BLOG_REPO)
    # git add .
    repo.git.add(all=True)
    # git commit -m "updates blog"
    repo.index.commit(commit_message)
    # git push
    origin = repo.remote(name='origin')
    origin.push()

In [102]:
random_text_string = "kdsaflkjiewoasvantluat"

In [103]:
with open(PATH_TO_BLOG/"index.html", 'w') as f:
    f.write(random_text_string)

In [104]:
update_blog()

In [105]:
import shutil
def create_new_blog(title,content,cover_image):
    cover_image= Path(cover_image)
    
    # auto file naming utility
    files = len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image,PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        # WRITE A NEW HTML FILE
        with open(path_to_new_content,"w") as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>")
            # OpenAI --->> Completion GPT -->> "hello\nblog post\n"
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog created")
            return path_to_new_content
    else:
        raise FileExistsError("File already exists, please check again your name! Aborting!")

In [106]:
path_to_new_content = create_new_blog('Test_title','thisisatest','templogo.png')

Blog created


In [107]:
# index.html --> Blog posts

In [108]:
from bs4 import BeautifulSoup as Soup

In [109]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [110]:
str(soup)

'kdsaflkjiewoasvantluat'

In [111]:
# checking duplicate links

In [112]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    # /pathtocontent/1.html check for overwriting
    return content_path in urls

In [113]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/"index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))
        

In [114]:
write_to_index(path_to_new_content)

IndexError: list index out of range

In [35]:
update_blog()

In [1]:
def create_prompt(title):
    prompt = """
    Biography: My name is Kristin and I am a Software Engineer with a emphasis on Devops practices.
    
    Blog
    Title: {}
    tags: tech, devops, AI, python, observability, coding
    Summary: I talk about what the future of AI could hold for Devops
    Full Text: """.format(title)
    return prompt

In [2]:
title = "The future of devops and AI"
print(create_prompt(title))


    Biography: My name is Kristin and I am a Software Engineer with a emphasis on Devops practices.
    
    Blog
    Title: The future of devops and AI
    tags: tech, devops, AI, python, observability, coding
    Summary: I talk about what the future of AI could hold for Devops
    Full Text: 


In [6]:
response = openai.Completion.create(engine='text-davinci-003',
                                    prompt=create_prompt(title),
                                    max_tokens=1000,
                                    temperature=0.7)

In [7]:
blog_content = response['choices'][0]['text']

In [8]:
print(blog_content)


    The future of DevOps and AI is an exciting prospect for the tech industry. With the ability to automate complex tasks, DevOps practitioners can focus on more strategic tasks and create more efficient workflows. AI can also be used to create predictive models that can improve the accuracy of decisions, as well as provide better insights into the performance of applications and services. By leveraging AI, DevOps teams can identify and address areas of inefficiency and improve the overall quality of the product.

AI and DevOps also go hand-in-hand when it comes to observability. By leveraging AI-driven observability solutions, DevOps teams can quickly identify issues in their environment and take corrective actions. AI-driven observability solutions are also able to detect anomalies in real-time, which helps teams stay ahead of problems and ensure that their systems remain stable.

Finally, AI can also be used to automate the coding process, which can help DevOps teams accelerate the

In [32]:
def dalle2_prompt(title):
    prompt = f"pixel digital art with no text showing {title}"
    return prompt

In [33]:
image_prompt =  dalle2_prompt(title)

In [34]:
image_prompt

'pixel digital art with no text showing The future of devops and AI'

In [35]:
response = openai.Image.create(prompt=image_prompt,
                              n=1,size="1024x1024")

In [36]:
image_url = response['data'][0]['url']

In [37]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-ZJxtr8k7uUYQka7iCmvXgZay/user-P0RAWNfvL0Wnea1OjOcqsSUc/img-Y9mJ1LfQWzQgFcsWutGp9650.png?st=2023-03-21T20%3A18%3A20Z&se=2023-03-21T22%3A18%3A20Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-03-21T16%3A41%3A18Z&ske=2023-03-22T16%3A41%3A18Z&sks=b&skv=2021-08-06&sig=SQyy/9UA/dXEgrtoJ4KZuFMtvddUjbaKIpoRgJCpW1c%3D'

In [28]:
from PIL import Image

ModuleNotFoundError: No module named 'PIL'

In [29]:
from pillow import Image

ModuleNotFoundError: No module named 'pillow'

In [30]:
from PIL import Image

In [31]:
def save_image(image_url, file_name):
    image_res = requests.get(image_url, stream = True)
    
    if image_res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(image_res.raw, f)
    else:
        print("Error downloading image!")
    return image_res.status_code

In [41]:
import requests
import shutil

In [42]:
save_image(image_url,file_name="title2.png")

200